In [41]:
import openai

client = openai.Client()

In [43]:
examples = [
    ("🚀 Just launched our biggest update yet! Try it now and let us know what you think!", "1"),
    ("The server maintenance window is scheduled for tonight from 11pm to 1am.", "0"),
    ("You won't believe what we just built! This is going to change everything. Check it out!", "1"),
    ("All employees must complete the safety training module by Friday.", "0"),
    ("🎉 We hit 500K users! Thank you to everyone who's been part of this journey!", "1"),
    ("The quarterly financial report has been uploaded to the shared drive.", "0"),
    ("Want to save 3 hours every day? Here's the secret workflow that changed my life!", "1"),
    ("Reminder: Parking lot will be repaved this weekend. Please move your vehicles.", "0"),
    ("🔥 Breaking: Our new AI feature is live! What do you think? Share your thoughts!", "1"),
    ("The company picnic is scheduled for next Saturday at Central Park.", "0"),
    ("This one trick will revolutionize how you work. Curious? Read on!", "1"),
    ("All department heads should submit their budget proposals by end of month.", "0"),
    ("💡 Pro tip: This simple change increased my productivity by 200%. Want to know how?", "1"),
    ("The office will be closed on Monday for the holiday.", "0"),
    ("We just released something amazing! Have you tried it? What's your experience?", "1"),
]
classes = list(set(cls for _, cls in examples))
print(classes)

['1', '0']


In [44]:
def llm(prompt):
    resp = client.chat.completions.create(model="gpt-4o", messages=[{"role":"user", "content": prompt}])
    return resp.choices[0].message.content

In [45]:
def evaluate(prompt, text):
    res = llm(prompt.format(text=text))
    for cls in classes:
        if cls in res.upper():
            return cls
    return "???"

In [46]:
bad_prompt = "Classify this text. Return 1 or 0 only. Text: {text}"

In [49]:
correct = []
for example, target in examples:
    res = evaluate(bad_prompt, example)
    if res != target:
        print(example, f"target: {target} result: {res}")
    correct.append(1 if res == target else 0)

print(f"{sum(correct)}/{len(correct)}")

The server maintenance window is scheduled for tonight from 11pm to 1am. target: 0 result: 1
All employees must complete the safety training module by Friday. target: 0 result: 1
Reminder: Parking lot will be repaved this weekend. Please move your vehicles. target: 0 result: 1
This one trick will revolutionize how you work. Curious? Read on! target: 1 result: 0
All department heads should submit their budget proposals by end of month. target: 0 result: 1
10/15


In [51]:
meta_prompt = f"""I use llm to classify texts. It made some mistakes. Please, improve my prompt using it's mistakes. <prompt>{bad_prompt}</prompt>. Here are mistakes: ```The server maintenance window is scheduled for tonight from 11pm to 1am. target: 0 result: 1
All employees must complete the safety training module by Friday. target: 0 result: 1
Reminder: Parking lot will be repaved this weekend. Please move your vehicles. target: 0 result: 1
This one trick will revolutionize how you work. Curious? Read on! target: 1 result: 0
All department heads should submit their budget proposals by end of month. target: 0 result: 1```. Return new prompt in <new_prompt> tag"""

In [52]:
res = llm(meta_prompt)

In [53]:
res

'<new_prompt>\nClassify the following text based on its nature: If the text is an announcement, reminder, or directive related to logistics or procedural matters (e.g., scheduling, deadlines), return 0. If the text offers advice or insights intended to evoke curiosity or inspire action, return 1. Return either 0 or 1 only. Text: {text}\n</new_prompt>'

In [55]:
new_prompt = "Classify the following text based on its nature: If the text is an announcement, reminder, or directive related to logistics or procedural matters (e.g., scheduling, deadlines), return 0. If the text offers advice or insights intended to evoke curiosity or inspire action, return 1. Return either 0 or 1 only. Text: {text}"

In [56]:
new_prompt

'Classify the following text based on its nature: If the text is an announcement, reminder, or directive related to logistics or procedural matters (e.g., scheduling, deadlines), return 0. If the text offers advice or insights intended to evoke curiosity or inspire action, return 1. Return either 0 or 1 only. Text: {text}'

In [57]:
correct = []
for example, target in examples:
    res = evaluate(new_prompt, example)
    if res != target:
        print(example, f"target: {target} result: {res}")
    correct.append(1 if res == target else 0)

print(f"{sum(correct)}/{len(correct)}")

15/15
